In [ ]:
from typing import List, Tuple, Dict, Any, Optional
from multiprocessing import Pool
from pandarallel import pandarallel
import os
import re
import sys
import json
from pathlib import Path

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import pandas as pd
from tqdm.auto import tqdm
tqdm.pandas()
pandarallel.initialize(progress_bar=True)

INFO: Pandarallel will run on 24 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [4]:
PATH_BUGS = "../reports/bugs_v001.csv"

In [ ]:
df_bugs = pd.read_csv(PATH_BUGS)
# drop github_link, crash_msg, substr_crash_msg
df_bugs = df_bugs.drop(columns=[
    'github_link', 'crash_msg', 'substr_crash_msg'])
# rename is_stil_triggerable to triggerable
df_bugs = df_bugs.rename(columns={"is_still_triggerable": "triggerable"})
# replace string values with tick marks (\ding{51}) if starting with yes, cross mark (\ding{55}) otherwise
df_bugs["triggerable"] = df_bugs["triggerable"].apply(
    lambda x: r"\ding{51}" if x.startswith("yes") else r"\ding{55}")
# go through all values and replace anything between backticks with \code{...}
df_bugs["short_desc"] = df_bugs["short_desc"].apply(lambda x: re.sub(
    r'`([^`]+)`', r'\\code{\1}', str(x)) if isinstance(x, str) else x)
# rename column by Capitalize and remove _
df_bugs.columns = df_bugs.columns.str.replace("_", " ").str.title()
# replace \n with " "
# df_bugs = df_bugs.applymap(lambda x: x.replace("\n", " "))
# start index at 1
df_bugs.index += 1
# print latex
print(df_bugs.to_latex(index=True))

\begin{tabular}{llllll}
\toprule
 & Platform & Short Desc & Status & Oracle & Triggerable \\
\midrule
1 & TKET & Implicit register reordering altering semantics & Confirmed & Equivalence & \ding{55} \\
2 & PennyLane & Unexpected reallocation of gates to different qubits & Confirmed & Equivalence & \ding{55} \\
3 & Qiskit & Missing standard \code{rxx} gate definition & Confirmed & Crash & \ding{55} \\
4 & BQSKit & Unrecognized delay gate & Confirmed & Crash & \ding{51} \\
5 & TKET & Error converting decomposed circuit (\code{PhasedX} op) & Confirmed & Crash & \ding{51} \\
6 & BQSKit & Missing controlled phase \code{cs} gate definition & Confirmed & Crash & \ding{51} \\
7 & PennyLane & Error exporting QASM file: \code{MidMeasureMP} op & Confirmed & Crash & \ding{51} \\
8 & PennyLane & Unitary matrix mismatch in \code{cu} gate implementation & Confirmed & Equivalence & \ding{51} \\
9 & TKET & Incompat. of \code{PhasedX} gate with \code{GreedyPauliSimp} opt & Confirmed & Equivalence & \din